In [1]:
import os
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import argparse
from model.classifier import VGG16
from model.evaluator import FaceNet
from model.SAC import Agent
from model.gan import Generator
from utils.mi_attack import inversion_dcgan, inversion_stylegan
from utils.file_utils import load_my_state_dict, low2high, seed_everything
import pickle
import warnings

In [2]:
MODEL_NAME = "VGG16"
MAX_EPISODES = 40000
MAX_STEP = 10
SEED = 42
ALPHA = 0.2
N_CLASSES = 1000
Z_DIM = 100
N_TARGET = 100

### load if generator should be stylegan2 trained on faces (ffhq)

In [5]:
print(f"Target Model: {MODEL_NAME}")
with open ('../checkpoints/gan/GeneratorStyleffhq.pkl', 'rb') as f:
    G = pickle.load(f)['G_ema'].cuda()
G.eval()
# G = Generator(Z_DIM)
# path_G = "checkpoints/gan/Genen.tar"
# G = torch.nn.DataParallel(G).cuda()
# G.load_state_dict(torch.load(path_G)['state_dict'], strict=False)

Target Model: VGG16


Generator(
  (synthesis): SynthesisNetwork(
    (b4): SynthesisBlock(
      (conv1): SynthesisLayer(
        (affine): FullyConnectedLayer()
      )
      (torgb): ToRGBLayer(
        (affine): FullyConnectedLayer()
      )
    )
    (b8): SynthesisBlock(
      (conv0): SynthesisLayer(
        (affine): FullyConnectedLayer()
      )
      (conv1): SynthesisLayer(
        (affine): FullyConnectedLayer()
      )
      (torgb): ToRGBLayer(
        (affine): FullyConnectedLayer()
      )
    )
    (b16): SynthesisBlock(
      (conv0): SynthesisLayer(
        (affine): FullyConnectedLayer()
      )
      (conv1): SynthesisLayer(
        (affine): FullyConnectedLayer()
      )
      (torgb): ToRGBLayer(
        (affine): FullyConnectedLayer()
      )
    )
    (b32): SynthesisBlock(
      (conv0): SynthesisLayer(
        (affine): FullyConnectedLayer()
      )
      (conv1): SynthesisLayer(
        (affine): FullyConnectedLayer()
      )
      (torgb): ToRGBLayer(
        (affine): FullyConn

### load target model

In [7]:
T = VGG16(N_CLASSES)
path_T = "../checkpoints/VGG16/nn_VGG16celeba.tar"
T = torch.nn.DataParallel(T).cuda()
T.load_state_dict(torch.load(path_T)['state_dict'], strict=False)
T.eval()

/home/bot/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/bot/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


DataParallel(
  (module): VGG16(
    (feature): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
      (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (9): ReLU(inplace=True)
      (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (12): ReLU(inplace=True)
      (13): MaxPool2d(kernel_size=2, stride=2, padding=

### load evaluator model

In [8]:
E = FaceNet(N_CLASSES)
path_E = '../checkpoints/VGG16/Evalceleba.tar'
E = torch.nn.DataParallel(E).cuda()
E.load_state_dict(torch.load(path_E)['state_dict'], strict=False)
E.eval()

DataParallel(
  (module): FaceNet(
    (feature): Backbone112(
      (input_layer): Sequential(
        (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): PReLU(num_parameters=64)
      )
      (output_layer): Sequential(
        (0): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): Dropout(p=0.5, inplace=False)
        (2): Flatten()
        (3): Linear(in_features=25088, out_features=512, bias=True)
        (4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (body): Sequential(
        (0): bottleneck_IR(
          (shortcut_layer): MaxPool2d(kernel_size=1, stride=2, padding=0, dilation=1, ceil_mode=False)
          (res_layer): Sequential(
            (0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (1): Conv2

In [9]:
seed_everything(seed=SEED)

In [10]:
total = 0
cnt = 0
cnt5 = 0

identities = range(N_CLASSES)
targets = random.sample(identities, N_TARGET)
os.makedirs("../attack_results/reinforcment_celebaStyle/images", exist_ok=True)
os.makedirs("../attack_results/reinforcment_celebaStyle/models", exist_ok=True)

### Attack Loop

In [11]:
for i in targets:
        agent = Agent(state_size=G.z_dim, action_size=G.z_dim, random_seed=SEED, hidden_size=256, action_prior="uniform")
        recon_image = inversion_stylegan(agent, G, T, ALPHA, z_dim=G.z_dim, max_episodes=MAX_EPISODES, max_step=MAX_STEP, label=i, model_name=MODEL_NAME)
        output= E(low2high(recon_image))
        eval_prob = F.softmax(output[0], dim=-1)
        top_idx = torch.argmax(eval_prob)
        _, top5_idx = torch.topk(eval_prob, 5)

        total += 1
        if top_idx == i:
            cnt += 1
        if i in top5_idx:
            cnt5 += 1

        acc = cnt / total
        acc5 = cnt5 / total
        print("Classes {}/{}, Accuracy : {:.3f}, Top-5 Accuracy : {:.3f}".format(total, N_TARGET, acc, acc5))

Target Label : 654
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
Episodes 10000/40000, Confidence score for the target model : 0.0016


FileNotFoundError: [Errno 2] No such file or directory: '../attack_results/reinforcment_celeba/imagesStyle/VGG16/final_image_10000_654.png'